In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2024-02-26 09:19:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2024-02-26 09:19:04 (564 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [4]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 20.1 MB/s eta 0:00:00


In [23]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

launch_site_options = []
for x in spacex_df['Launch Site'].unique():
  launch_site_options.append({'label': x, 'value': x})
launch_site_options.append({'label': 'All Sites', 'value': 'ALL'})
# launch_site_options

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options= launch_site_options,
                                value= 'ALL',
                                placeholder='Select a launch site',
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000,step=1000,
                                marks={0: '0', 1000:'1000', 2000:'2000', 3000:'3000', 4000:'4000', 5000:'5000',
                                       6000:'6000', 7000:'8000', 9000:'9000', 10000:'10000'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['class'] == 1]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df,
                     values='class',
                     names='Launch Site',
                     title='Total Successful Launches by All Sites')
        return fig
    else:
        specific = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(specific,
                     names='class',
                     title='Launch Outcome by' + entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'),
              Input(component_id='payload-slider',component_property='value'))

def get_scatter_slider_chart(entered_site, payload_slider):
  low_payload, high_payload = payload_slider
  filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > low_payload) & (spacex_df['Payload Mass (kg)'] < high_payload)]

  if entered_site == 'ALL':
    fig = px.scatter(filtered_df,
                      x='Payload Mass (kg)',
                      y='class',
                      color='Booster Version Category',
                      title='Relation b/w Payload Mass and Mission Outcomes for All Sites')
    return fig
  else:
    fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
                      x='Payload Mass (kg)',
                      y='class',
                      color='Booster Version Category',
                      title='Relation b/w Payload Mass and Mission Outcomes for' + entered_site)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>